# What is the doggest dog?

This is a notebook that can be used to perform experiments connected to our paper.

The aim of our method is to determine prototypes and anti-prototypes for a given basic-level category. Our methods aim to examine deep learning models from the perspective of the Prototype Theory.

Method from the paper: Strategy 2 (based on within-category similarity AND dissimilarity to non-members)

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

We use the following function to find the hyponyms for a given basic-level category - hypernym:

In [3]:
def find_hyponyms(hypernym_name):
    """
    Function ca be used to obtain a list of indexes of the ImageNet classes 
    belonging to a given basic-level category (a hypernym).
    ------
    Pramaters: 
    
    hypernym_name: a desired hypernym (e.g. "domestic_cat")
    """
    from nltk.corpus import wordnet as wn
    import nltk
    from tensorflow.keras.applications.imagenet_utils import decode_predictions
    from tensorflow.keras.utils import to_categorical
    import numpy as np
    
    hyponyms = wn.synsets(hypernym_name)[0]
    hyponyms = set([i for i in hyponyms.closure(lambda s:s.hyponyms())])
    offsets = []
    imagenet_classes = decode_predictions(to_categorical(np.expand_dims(np.array(range(1000)), axis=-1), num_classes=1000), top=1)

    for c in imagenet_classes:
        offsets.append(int(c[0][0].split('n')[1]))
    
    ids = []
    for idx, o in enumerate(offsets):
        isadoggo = wn.synset_from_pos_and_offset('n', int(o))
        if isadoggo in hyponyms:
            ids.append(idx)
    return np.array(ids)

In [4]:
high_level_category = find_hyponyms(hypernym_name='animal')

indexes of all animals in ImageNet:

In [5]:
high_level_category

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [6]:
def return_prototype_antiprototype(model, hypernym_name, high_level_category, keras=True, levit=False):
    """
    Function can be used to find prototypes and anti-prototypes for a given basic-level category 
    for a given deep learning model. 
    ------
    Parameters: 
    
    model: keras/torch model trained on ImageNet (with a standard order of classes that can be decoded with 
    tensorflow.keras.applications.imagenet_utils.decode_predictions
    hypernym_name: string with a name of a basic-level category, e.g. "domestic_cat", "dog" etc.
    high_level_category: in strategy 2, besides the hyponyms of our desired category, we also use
    a contrasting category. A contrasting category is a complement of a set of some higher-level categories. 
    high_level_category is a list with ImageNet indexes of this higher-level category (its values are determined above).
    keras: flag whether we want to use a keras model (set True if yes - it is a default). 
    Set false in the case of using a torch model.
    levit: set with keras=False in the case of using one of the LeViTs models.
    """
    from tensorflow.keras.applications.imagenet_utils import decode_predictions
    from sklearn.metrics.pairwise import cosine_similarity
    from tensorflow.keras.utils import to_categorical

    all_classes = high_level_category
    
    print("All classes taken into consideration - members of the examined category + members of the contrasting category:")
    print(all_classes)
    
    hyponym_classes = find_hyponyms(hypernym_name=hypernym_name)
    dissimilar_classes = list(set(all_classes) - set(hyponym_classes))
    
    print(f"Members of the same category (for hypernym = {hypernym_name}):")
    print(len(hyponym_classes))
    print(f"Members of the contrasting category (contrasting to {hypernym_name}):")
    print(len(dissimilar_classes))
    
    # Extraction of weights (we consider models from Keras Application (https://keras.io/api/applications/)
    # and HuggingFace (https://huggingface.co/models) listed in the provided file Model_List.pdf
    if keras:
        hyponym_weights = np.mean(model.layers[-1].get_weights()[0][:, hyponym_classes], axis=1)
        contrasting_hyponym_weights = np.mean(model.layers[-1].get_weights()[0][:, dissimilar_classes], axis=1)
        hyponyms = np.moveaxis(model.layers[-1].get_weights()[0][:, hyponym_classes], -1, 0)
        hypernym_template = np.expand_dims(hyponym_weights, axis=0)
        hypernym_template_contrasting = np.expand_dims(contrasting_hyponym_weights, axis=0)
    else:
        if not levit:
            hyponym_weights = np.mean(model.classifier.weight.detach().numpy()[hyponym_classes, :], axis=0)
            contrasting_hyponym_weights = np.mean(model.classifier.weight.detach().numpy()[dissimilar_classes, :], axis=0)
            hyponyms = model.classifier.weight.detach().numpy()[hyponym_classes, :]
            hypernym_template = np.expand_dims(hyponym_weights, axis=0)
            hypernym_template_contrasting = np.expand_dims(contrasting_hyponym_weights, axis=0)
        else:
            # models from the LeViT family have slightly different model structure than the other models 
            # from HuggingFace
            hyponym_weights = np.mean(model.classifier_distill.linear.weight.detach().numpy()[hyponym_classes, :], axis=0)
            contrasting_hyponym_weights = np.mean(model.classifier_distill.linear.weight.detach().numpy()[dissimilar_classes, :], axis=0)
            hyponyms = model.classifier_distill.linear.weight.detach().numpy()[hyponym_classes, :]
            hypernym_template = np.expand_dims(hyponym_weights, axis=0)
            hypernym_template_contrasting = np.expand_dims(contrasting_hyponym_weights, axis=0)
        
    cs_members = np.squeeze(cosine_similarity(hyponyms, hypernym_template))
    cs_contrasting = np.squeeze(cosine_similarity(hyponyms, hypernym_template_contrasting))
    
    # Calculating the value of our final Score to find prototypes and anti-types
    squeezed_cs = np.squeeze(cs_members - cs_contrasting)
    
    print("prototype")
    most_similar = decode_predictions(np.expand_dims(to_categorical(hyponym_classes[np.argmax(squeezed_cs)], num_classes=1000), axis=0), top=1)[0][0]
    print(most_similar)
    print("anti-prototype")
    least_similar = decode_predictions(np.expand_dims(to_categorical(hyponym_classes[np.argmin(squeezed_cs)], num_classes=1000), axis=0), top=1)[0][0]
    print(least_similar)
    print()
    print("=========================================")
    print()
    return most_similar[1], least_similar[1]

Below one needs to set a WordNet node name - a desired basic-level category.

**In our experiments, we use the following nodes:**

Natural categories:

* *domestic_cat*
* *dog*
* *bird*
* *fish*
* *mammal*

Artificial category: *musical_instrument*

In [7]:
hypernym_name = "domestic_cat"

Below, we present how we gathered the results for our paper (for category domestic_cat as an example):

In [8]:
names_list = []
most_similar_list = []
least_similar_list = []

In [9]:
# generating results for different CNNs (keras)

# ---------------------------------------------------------------------
model = tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.VGG19(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ResNet101(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ResNet101V2(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ResNet152(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)


print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ResNet152V2(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.MobileNetV2(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.DenseNet169(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.DenseNet201(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.NASNetMobile(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.NASNetLarge(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.EfficientNetV2B0(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)


print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.EfficientNetV2B1(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)


print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.EfficientNetV2B2(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.EfficientNetV2B3(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.EfficientNetV2S(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ConvNeXtTiny(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------------
model = tf.keras.applications.ConvNeXtSmall(
    include_top=True,
    weights="imagenet",
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)

print(model.name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=True)

names_list.append(model.name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)




inception_v3
All classes taken into consideration - members of the examined category + members of the contrasting category:
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 21

In [10]:
# generating results for different transformers (pytorch)

from transformers import AutoModelForImageClassification

name = "microsoft/swinv2-tiny-patch4-window8-256"
model = AutoModelForImageClassification.from_pretrained(name)

print(name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

from transformers import MobileViTForImageClassification

name = "apple/mobilevit-small"
model = MobileViTForImageClassification.from_pretrained(name)

print(name)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------
from transformers import MobileViTForImageClassification

name = "apple/mobilevit-xx-small"
model = MobileViTForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------
from transformers import SwinForImageClassification

name = "microsoft/swin-base-patch4-window7-224"
model = SwinForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------
from transformers import SwinForImageClassification

# name = "microsoft/swin-large-patch4-window7-224"
name = "microsoft/swin-base-patch4-window12-384"
model = SwinForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)



# ---------------------------------------------------------------
from transformers import SwinForImageClassification

name = "microsoft/swin-tiny-patch4-window7-224"
model = SwinForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------
from transformers import ViTForImageClassification

name = 'facebook/deit-base-patch16-224'
model = ViTForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------
from transformers import ViTForImageClassification

name = 'facebook/deit-small-patch16-224'
model = ViTForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = "microsoft/swinv2-small-patch4-window16-256"
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = "microsoft/swinv2-base-patch4-window16-256"
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'facebook/deit-base-patch16-384'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)


names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'facebook/deit-tiny-patch16-224'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'microsoft/cvt-13'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'microsoft/cvt-21'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'facebook/levit-128S'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False, levit=True)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'facebook/levit-128'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False, levit=True)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'facebook/levit-192'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False, levit=True)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'facebook/levit-256'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False, levit=True)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

name = 'facebook/levit-384'
model = AutoModelForImageClassification.from_pretrained(name)

print(model.__class__.__name__)
most_similar, least_similar = return_prototype_antiprototype(model=model, high_level_category=high_level_category, 
                                                        hypernym_name=hypernym_name, 
                                                        keras=False, levit=True)

names_list.append(name)
most_similar_list.append(most_similar)
least_similar_list.append(least_similar)

# ---------------------------------------------------------------

microsoft/swinv2-tiny-patch4-window8-256
All classes taken into consideration - members of the examined category + members of the contrasting category:
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 

In [11]:
df = pd.DataFrame(data={"Model_name":names_list, 
                   "Most_similar":most_similar_list , 
                   "Least_similar":least_similar_list})

In [12]:
df

,Model_name,Most_similar,Least_similar
0,inception_v3,tabby,Siamese_cat
1,xception,tabby,Siamese_cat
2,vgg16,tabby,Siamese_cat
3,vgg19,tabby,Siamese_cat
4,resnet50,tabby,Siamese_cat
5,resnet50v2,tabby,Siamese_cat
6,resnet101,tabby,Siamese_cat
7,resnet101v2,tabby,Siamese_cat
8,resnet152,tabby,Siamese_cat
9,resnet152v2,tabby,Siamese_cat


In [13]:
df.Most_similar.value_counts()

Most_similar
tabby           37
Egyptian_cat     5
Name: count, dtype: int64

In [14]:
df.Least_similar.value_counts()

Least_similar
Siamese_cat    38
Persian_cat     4
Name: count, dtype: int64